데이터 전처리

In [8]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath("../"))

from utils import *
from MLP import train_and_evaluate
from permutation import permutation

# Load dataset
region="seoul"
path = f"../../collect_data/filtered/kma/merged/kma_{region}_meta.csv"

# 2. CSV 파일 읽어서 데이터프레임으로 변환
korea_data = pd.read_csv(path)
china_data = concat_china_data(korea_data)

korea_data = convert_timesteps(korea_data)
print_missing_info(korea_data)

china_data = convert_timesteps(china_data)
print_missing_info(china_data)

(40175, 26)
Series([], dtype: float64)
Index(['TM', 'STN', 'WD', 'WS', 'PA', 'PS', 'TA', 'TD', 'HM', 'PV', 'WW',
       'CA_TOT', 'CA_MID', 'CT', 'VS', 'TS', 'TE_005', 'TE_01', 'TE_02',
       'TE_03', 'IR', 'PM10', '지점', '위도', '경도', 'diff'],
      dtype='object')
(9488, 111)
VV             0.031619
WD_yanan       0.021079
WS_yanan       0.021079
TA_yanan       0.105396
TD_yanan       0.274030
HM_yanan       0.685076
PS_yanan       0.031619
PR             0.063238
VV_tongliao    0.010540
WD_tongliao    0.010540
WS_tongliao    0.010540
TA_tongliao    0.137015
TD_tongliao    0.252951
HM_tongliao    0.242411
PS_tongliao    0.010540
PR_tongliao    0.010540
VV_qingdao     0.094857
WD_qingdao     0.115936
WS_qingdao     0.115936
TA_qingdao     0.094857
TD_qingdao     0.200253
HM_qingdao     0.790472
PS_qingdao     0.094857
PR_qingdao     0.094857
VV_chifeng     0.063238
WD_chifeng     0.021079
WS_chifeng     0.063238
TA_chifeng     0.115936
TD_chifeng     0.263491
HM_chifeng     0.179174
PS_

MLP-Model - 한국만

In [9]:
removed_columns=['TM', 'PM10','WW','CT','diff']

# Features (X) and target (y)
X = korea_data.drop(columns=removed_columns)

# 결측값 처리하는 세가지 방안
#X.fillna(0, inplace=True)
X = X.fillna(X.mean())
#X.fillna(X.median(), inplace=True)

y = korea_data['PM10']

# Train the model and get the necessary data
mean_mae, test_mae, trained_model, X_train_val, y_train_val, X_test, y_test = train_and_evaluate(
    X, y, test_size=0.2, n_splits=5, epochs=100, batch_size=64
)



Cross-validated Mean Absolute Error: 21.225131225585937
252/252 [==============================] - 0s 500us/step - loss: 540.9431 - mae: 16.4465
Test Set Mean Absolute Error: 16.44645881652832


질적 분석

In [10]:
# Evaluate the model and calculate permutation importance
mean_mae, low_importance_features_korea, features_with_scores = permutation(
    trained_model, X_train_val, y_train_val, X_test, y_test
)

Model Evaluation - Mean Absolute Error: 16682.224609375
252/252 [==============================] - 0s 435us/step
Feature: STN, Importance: 0.0
Feature: WD, Importance: 0.892429265546889
Feature: WS, Importance: -0.026680309668154222
Feature: PA, Importance: 32.47354513472019
Feature: PS, Importance: 358.252461261881
Feature: TA, Importance: 2.4366405256259895
Feature: TD, Importance: -1.9825400708268717
Feature: HM, Importance: 24.64650092100237
Feature: PV, Importance: -0.010264513955189614
Feature: CA_TOT, Importance: -0.20074005523456434
Feature: CA_MID, Importance: 0.8379470295134525
Feature: VS, Importance: 393.01970587738214
Feature: TS, Importance: -5.201222012698054
Feature: TE_005, Importance: 7.940764746498462
Feature: TE_01, Importance: -2.456393461276457
Feature: TE_02, Importance: -2.205290079715269
Feature: TE_03, Importance: -0.1438111732732068
Feature: IR, Importance: -0.07916588109728764
Feature: 지점, Importance: 0.0
Feature: 위도, Importance: 0.0
Feature: 경도, Importance:

MLP model - 중국 데이터 포함

In [11]:
removed_columns=['TM', 'PM10','WW','CT','STN_dalian','STN_tongliao','STN_qingdao','STN_chifeng','diff']

# Features (X) and target (y)
X = china_data.drop(columns=removed_columns)

# 결측값 처리하는 세가지 방안
#X.fillna(0, inplace=True)
X = X.fillna(X.mean())
#X.fillna(X.median(), inplace=True)

y = china_data['PM10']

# Train the model and get the necessary data
mean_mae, test_mae, trained_model, X_train_val, y_train_val, X_test, y_test = train_and_evaluate(
    X, y, test_size=0.2, n_splits=5, epochs=100, batch_size=64
)

Cross-validated Mean Absolute Error: 24.38521385192871
60/60 [==============================] - 0s 483us/step - loss: 657.0121 - mae: 18.5322
Test Set Mean Absolute Error: 18.532236099243164


질적 분석

In [12]:
# Evaluate the model and calculate permutation importance
mean_mae, low_importance_features_china, features_with_scores = permutation(
    trained_model, X_train_val, y_train_val, X_test, y_test
)

Model Evaluation - Mean Absolute Error: 70275.2890625
60/60 [==============================] - 0s 474us/step
Feature: STN, Importance: 0.0
Feature: WD, Importance: 0.00013624539715237916
Feature: WS, Importance: 0.00017987684841500596
Feature: PA, Importance: 0.0002484111610101536
Feature: PS, Importance: 0.00025684932188596576
Feature: TA, Importance: 0.0003276475297752768
Feature: TD, Importance: 0.000272696596221067
Feature: HM, Importance: 0.00023215227265609427
Feature: PV, Importance: 0.0002375033000134863
Feature: CA_TOT, Importance: 0.00022062698117224498
Feature: CA_MID, Importance: 0.0001257491487194784
Feature: VS, Importance: 0.00020375066524138674
Feature: TS, Importance: 0.0002916310011642054
Feature: TE_005, Importance: 0.00017658391006989405
Feature: TE_01, Importance: 0.0002058087513432838
Feature: TE_02, Importance: 0.00022721286222804338
Feature: TE_03, Importance: 0.00026322939229430633
Feature: IR, Importance: 0.00013727444020332767
Feature: 지점, Importance: 0.0
Fea

## MLP 한국만
Feature Importance 음수인 feature 제거한 버전

In [13]:
print(low_importance_features_korea)

['STN', 'WS', 'TD', 'PV', 'CA_TOT', 'TS', 'TE_01', 'TE_02', 'TE_03', 'IR', '지점', '위도', '경도']


In [14]:
removed_columns=['TM', 'PM10','WW','CT','diff']
removed_columns+=low_importance_features_korea

# Features (X) and target (y)
X = korea_data.drop(columns=removed_columns)

# 결측값 처리하는 세가지 방안
#X.fillna(0, inplace=True)
X = X.fillna(X.mean())
#X.fillna(X.median(), inplace=True)

y = korea_data['PM10']

# Train the model and get the necessary data
mean_mae, test_mae, trained_model, X_train_val, y_train_val, X_test, y_test = train_and_evaluate(
    X, y, test_size=0.2, n_splits=5, epochs=100, batch_size=64
)

Cross-validated Mean Absolute Error: 22.05750389099121
252/252 [==============================] - 0s 460us/step - loss: 533.8541 - mae: 16.5226
Test Set Mean Absolute Error: 16.52264404296875


In [15]:
# Evaluate the model and calculate permutation importance
mean_mae, low_importance_features, features_with_scores = permutation(
    trained_model, X_train_val, y_train_val, X_test, y_test
)

Model Evaluation - Mean Absolute Error: 19691.361328125
252/252 [==============================] - 0s 384us/step
Feature: WD, Importance: 0.5243554569758999
Feature: PA, Importance: 18.23132279962774
Feature: PS, Importance: 258.5967186165799
Feature: TA, Importance: 0.776996703893019
Feature: HM, Importance: 16.89805437433315
Feature: CA_MID, Importance: 0.4296941013843025
Feature: VS, Importance: 273.76774021816857
Feature: TE_005, Importance: -0.3643499339112168

Top 10 Important Features (Sorted Descending by Importance Score):
Feature: VS, Importance: 273.76774021816857
Feature: PS, Importance: 258.5967186165799
Feature: PA, Importance: 18.23132279962774
Feature: HM, Importance: 16.89805437433315
Feature: TA, Importance: 0.776996703893019
Feature: WD, Importance: 0.5243554569758999
Feature: CA_MID, Importance: 0.4296941013843025
Feature: TE_005, Importance: -0.3643499339112168


## MLP 중국포함
Feature Importance 음수인 feature 제거한 버전

In [16]:
print(low_importance_features_china)

['STN', '지점', '위도', '경도', 'STN_yanan', 'IW', 'IR_yanan', 'IX', 'RH', 'LON', 'LAT', 'IW_tongliao', 'IR_tongliao', 'IX_tongliao', 'RH_tongliao', 'LON_tongliao', 'LAT_tongliao', 'IW_qingdao', 'IR_qingdao', 'IX_qingdao', 'RH_qingdao', 'LON_qingdao', 'LAT_qingdao', 'IW_chifeng', 'IR_chifeng', 'IX_chifeng', 'RH_chifeng', 'LON_chifeng', 'LAT_chifeng', 'IW_dalian', 'IR_dalian', 'IX_dalian', 'RH_dalian', 'LON_dalian', 'LAT_dalian']


In [17]:
removed_columns=['TM', 'PM10','WW','CT','STN_dalian','STN_tongliao','STN_qingdao','STN_chifeng','diff']
removed_columns+=low_importance_features_china

# Features (X) and target (y)
X = china_data.drop(columns=removed_columns)

# 결측값 처리하는 세가지 방안
#X.fillna(0, inplace=True)
X = X.fillna(X.mean())
#X.fillna(X.median(), inplace=True)

y = china_data['PM10']

# Train the model and get the necessary data
mean_mae, test_mae, trained_model, X_train_val, y_train_val, X_test, y_test = train_and_evaluate(
    X, y, test_size=0.2, n_splits=5, epochs=100, batch_size=64
)

Cross-validated Mean Absolute Error: 24.8831844329834
60/60 [==============================] - 0s 492us/step - loss: 626.5782 - mae: 17.7830
Test Set Mean Absolute Error: 17.78302574157715


In [20]:
# Evaluate the model and calculate permutation importance
mean_mae, low_importance_features, features_with_scores = permutation(
    trained_model, X_train_val, y_train_val, X_test, y_test
)

Model Evaluation - Mean Absolute Error: 16866.869140625
60/60 [==============================] - 0s 441us/step
Feature: WD, Importance: 0.396114828159989
Feature: WS, Importance: -0.005603220290868194
Feature: PA, Importance: 118.72996531768732
Feature: PS, Importance: 76.14634407041267
Feature: TA, Importance: 1.8855956661918754
Feature: TD, Importance: 0.030477570380753605
Feature: HM, Importance: -4.90375360020771
Feature: PV, Importance: 1.0024736288338318
Feature: CA_TOT, Importance: 0.0663941434602748
Feature: CA_MID, Importance: 0.2728623382146907
Feature: VS, Importance: 197.06464962496867
Feature: TS, Importance: -5.87845413406103
Feature: TE_005, Importance: 5.988997573619054
Feature: TE_01, Importance: -1.1814258265938407
Feature: TE_02, Importance: -1.6984639464251814
Feature: TE_03, Importance: -1.075236018416399
Feature: IR, Importance: 0.05238622056313034
Feature: VV, Importance: 0.0025189326341205744
Feature: WD_yanan, Importance: 1.179614876744381
Feature: WS_yanan, Im